## EKS 환경 설정

아래 노트북은 원본 노트북에 설명을 추가 하였습니다.
- 실제 실행시에 여기의 셀의 "결과 값" 과 일치하는지를 확인하면서 실행 하십시오.
- 만일 에러등이 발생하면 확인하시고 다시 실행 해야 합니다.
    - https://github.com/data-science-on-aws/workshop/blob/master/12_kubeflow/00_01_Configure_Environment.ipynb

# Associate IAM Policies with EKS Worker Nodes


In [2]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD
export INSTANCE_PROFILE_ARN=arn:aws:iam::343441690612:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD


# Attach Roles

In [3]:
%%bash

### Source the environment
source ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take a few minutes.  Please be patient._


In [4]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve

echo "Completed"

[ℹ]  eksctl version 0.25.0
[ℹ]  using region ap-northeast-2
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 46.193815ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 107.984398ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 161.901324ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 264.566232ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 840.0924ms
[!]  ret

In [5]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.ap-northeast-2.amazonaws.com/id/89C372954BDCA6AC1A2F60DCFEF39EAD
Completed


# Update `~/.kube/config` with our new EKS cluster

In [6]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Updated context arn:aws:eks:ap-northeast-2:343441690612:cluster/cluster in /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [7]:
%%bash

kubectl get ns


NAME              STATUS   AGE
anonymous         Active   3h59m
cert-manager      Active   4h13m
default           Active   4h32m
istio-system      Active   4h13m
knative-serving   Active   4h12m
kube-node-lease   Active   4h32m
kube-public       Active   4h32m
kube-system       Active   4h32m
kubeflow          Active   4h13m


In [8]:
%%bash

kubectl get nodes


NAME                                                STATUS   ROLES    AGE     VERSION
ip-192-168-39-223.ap-northeast-2.compute.internal   Ready    <none>   4h27m   v1.17.9-eks-4c6976
ip-192-168-74-46.ap-northeast-2.compute.internal    Ready    <none>   4h27m   v1.17.9-eks-4c6976
